# Análise do vestibular ENEM 2019

## Introduction to Data Science

## The CRISP-DM approach to Data Science Tasks

_________________________

## I. Business Understanding
                    
#### The Business Understanding phase focuses on understanding the objectives and requirements of the project.  
+ Determine business objectives:   
+ Assess situation:  
+ Determine data mining goals:  
+ Produce project plan:  



#### Determine business objectives:   

* Compreender quais aspectos melhor determinam a nota de um aluno no Enem de acordo com a Teoria de Resposta ao Item.

- Observar se existe uma correlação entre as notas de cada uma das 4 áreas do conhecimento e a redação. 

* Entender como é distribuida a educação no país, e quais fatores interferem no desempenho de um estudante.

- Analisar se com as cotas há um acesso mais democrático às instituições públicas de ensino.

#### Assess situation:  

* Sabemos que o ENEM utiliza o TRI (Teoria de Resposta ao Item) que classifica as questões conforme a dificuldade, isto é, cada questão possui um peso pré-determinado que não conhecido do público. Esse sistema é utilizado para evitar "chutes", se um aluno erra uma questão considerada fácil e acerta uma questão considerada difícil, o aluno sofre uma penalidade. Desse modo, mesmo com o gabarito dos alunos não nos é possível determinar a nota correta dos alunos, pretendemos obter o melhor modelo de regrassão para atribuir a nota aos alunos.

- A priori, acreditamos que no Brasil, as escolas privadas e escolas públicas federais obtém um melhor desempenho no ENEM, do que os alunos de escolas públicas estaduais e municipais. Além disso, acreditamos que a locaclização geográfica da escola e da moradia do estudante e a renda familiar influencia no seu desempenho, alunos que moram em situações vuneráveis e|ou possuem baixa renda tendem a obter um desempenho inferior a média nacional. 

#### Determine data mining goals:  

- Obter os dados oficiais do INEP de todos os alunos que fizeram o vestibular no ano de 2019: https://www.gov.br/inep/pt-br/acesso-a-informacao/dados-abertos/microdados/enem onde consta os microdados do ENEM ("Os microdados do Enem são o menor nível de desagregação de dados recolhidos por meio do exame. Eles atendem a demanda por informações específicas ao disponibilizar as provas, os gabaritos, as informações sobre os itens, as notas e o questionário respondido pelos inscritos no Enem" - INEP).

* Obter as notas de corte de algumas universidades públicas brasileiras para analizar a democratização do ensino público.

#### Produce project plan: 

- Limpar os dados, por exemplo, remover os alunos treineiros e alunos que não concluiram o vestibular da base de dados.

* Analisar os nossos dados, obter as médias, os mínimos e os máximos.

- Criar classificadores, se necessário. 

* Determinar as informações que serão utilizadas em cada modelo.

- Observar se existe alguma correlação dentre as informações disponíveis. 

* Encontrar o melhor modelo de regressão para as notas.

- Analisar a questão das cotas e da nota de corte das universidades públicas.

* Reunir as informações obtidas e concluir o trabalho.


## II. Data Understanding  

+ Collect initial data:  
+ Describe data:  
+ Explore data:  
+ Verify data quality:  

Os dados oficiais do vestibular foram obtidos através de https://www.gov.br/inep/pt-br/acesso-a-informacao/dados-abertos/microdados/enem nele vamos utilizar o arquivo MICRODADDOS_ENEM_2019.csv com os dados brutos de todos os alunos que realizaram o ENEM. Já o arquivo Dicionário_Microdados_Enem_2019.xlsx descreve esses dados.

In [5]:
# Importando Bibliotecas 

import numpy as np
import pandas as pd
import dask
import json
import dask.dataframe as dd

# Importando os Dados

dic = pd.read_excel('Dicionário_Microdados_Enem_2019.xlsx' )

#df = pd.read_csv("MICRODADOS_ENEM_2019.csv",sep=";",encoding = "ISO-8859-1",decimal=",")
#df.head()

df = dd.read_csv("MICRODADOS_ENEM_2019.csv",sep=";",encoding = "ISO-8859-1",dtype={'NU_IDADE': 'float64'})

df.head()

,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,190001595656,2019,3552205,Sorocaba,35,SP,36.0,M,1,3,...,A,A,A,A,A,A,C,A,C,B
1,190001421546,2019,2910800,Feira de Santana,29,BA,23.0,M,1,1,...,A,A,A,B,A,A,E,A,A,B
2,190001133210,2019,2304400,Fortaleza,23,CE,39.0,F,1,3,...,A,A,A,B,A,A,C,A,A,B
3,190001199383,2019,1721000,Palmas,17,TO,25.0,F,1,1,...,A,A,A,C,A,B,D,A,B,B
4,190001237802,2019,3118601,Contagem,31,MG,22.0,F,1,1,...,B,A,A,B,A,A,C,B,B,B


### Observações gerais:

* Há 3 tipos de vestibulandos que realizaram o ENEM, os estudantes que ainda não terminaram o Ensino MÉdio, os alunos que concluiram o Ensino Médio no ano de 2019, e os alunos que concluiram o ENEM anteriormente
    - Segundo informações do INEP, as notas dos alunos em cada uma das provas segue uma distribuição normal com média 500 e desvião padrão 100 dos alunos que terminaram o vestibular no ano de 2019.
* As primeiras colunas referem-se aos dados pessoais do vestibulando:
    - Município de Residência;
    - Idade;
    - Sexo (M para masculino e F para feminino);
    - Estado Civil;
    - Raça; 
    - Nacionadlidade e cidade de nascença. 
* Em seguida há colunas referentes a escolaridade básica:
    - Se concluiu ou não o Ensino Médio;
    - Se sim em qual escola;
    - O ano que ele concluiu;
    - Informações da localização da escola;
    - Informações do tipo de escola.
* As próximas colunas se referem a necessidade de um atendimento especial por causa de alguma deficiência.
    - Não iremos entrar muito em detalhes pois nesse trabalho vamos focar nos estudadntes que não possuem problemas físicos.
* Depois, seguem as colunas com informações do local de aplicação do vestibular.
* Nas colunas posteriores os dados são relacionados à prova:
    - A presença dos alunos em cada umas das provas;
    - As repostas dos candidatos as questões objetivas;
    - Notas de cada uma das avaliações;
    - Gabarito oficial da prova que o aluno concluiu;
    - Nota da Redação e de cada uma das 5 competências.
* Por fim temos o questionário socioeconômico:
    - Perguntas aos candidatos sobre a sua família;
    - Perguntas sobre a renda familiar;
    - Perguntas sobre a moradia.

In [3]:
df.describe(include='all')


,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,CO_MUNICIPIO_NASCIMENTO,NO_MUNICIPIO_NASCIMENTO,CO_UF_NASCIMENTO,SG_UF_NASCIMENTO,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,CO_ESCOLA,CO_MUNICIPIO_ESC,NO_MUNICIPIO_ESC,CO_UF_ESC,SG_UF_ESC,TP_DEPENDENCIA_ADM_ESC,TP_LOCALIZACAO_ESC,TP_SIT_FUNC_ESC,IN_BAIXA_VISAO,IN_CEGUEIRA,IN_SURDEZ,IN_DEFICIENCIA_AUDITIVA,IN_SURDO_CEGUEIRA,IN_DEFICIENCIA_FISICA,IN_DEFICIENCIA_MENTAL,IN_DEFICIT_ATENCAO,IN_DISLEXIA,IN_DISCALCULIA,IN_AUTISMO,IN_VISAO_MONOCULAR,IN_OUTRA_DEF,IN_GESTANTE,IN_LACTANTE,IN_IDOSO,IN_ESTUDA_CLASSE_HOSPITALAR,IN_SEM_RECURSO,IN_BRAILLE,IN_AMPLIADA_24,IN_AMPLIADA_18,IN_LEDOR,IN_ACESSO,IN_TRANSCRICAO,IN_LIBRAS,IN_TEMPO_ADICIONAL,IN_LEITURA_LABIAL,IN_MESA_CADEIRA_RODAS,IN_MESA_CADEIRA_SEPARADA,IN_APOIO_PERNA,IN_GUIA_INTERPRETE,IN_COMPUTADOR,IN_CADEIRA_ESPECIAL,IN_CADEIRA_CANHOTO,IN_CADEIRA_ACOLCHOADA,IN_PROVA_DEITADO,IN_MOBILIARIO_OBESO,IN_LAMINA_OVERLAY,IN_PROTETOR_AURICULAR,IN_MEDIDOR_GLICOSE,IN_MAQUINA_BRAILE,IN_SOROBAN,IN_MARCA_PASSO,IN_SONDA,IN_MEDICAMENTOS,IN_SALA_INDIVIDUAL,IN_SALA_ESPECIAL,IN_SALA_ACOMPANHANTE,IN_MOBILIARIO_ESPECIFICO,IN_MATERIAL_ESPECIFICO,IN_NOME_SOCIAL,CO_MUNICIPIO_PROVA,NO_MUNICIPIO_PROVA,CO_UF_PROVA,SG_UF_PROVA,TP_PRESENCA_CN,TP_PRESENCA_CH,TP_PRESENCA_LC,TP_PRESENCA_MT,CO_PROVA_CN,CO_PROVA_CH,CO_PROVA_LC,CO_PROVA_MT,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,TX_RESPOSTAS_CN,TX_RESPOSTAS_CH,TX_RESPOSTAS_LC,TX_RESPOSTAS_MT,TP_LINGUA,TX_GABARITO_CN,TX_GABARITO_CH,TX_GABARITO_LC,TX_GABARITO_MT,TP_STATUS_REDACAO,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO,Q001,Q002,Q003,Q004,Q005,Q006,Q007,Q008,Q009,Q010,Q011,Q012,Q013,Q014,Q015,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
npartitions=1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,float64,float64,float64,object,float64,object,float64,object,float64,float64,float64,float64,object,float64,object,float64,float64,float64,float64,float64,float64,float64,object,float64,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,object,float64,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,object,object,object,object,float64,object,object,object,object,float64,float64,float64,float64,float64,float64,float64,object,object,object,object,float64,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [4]:
df.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 136 entries, NU_INSCRICAO to Q025
dtypes: object(41), float64(25), int64(70)

Analizando os dados faltates do dataframe:

In [5]:
nans = df.isnull().sum()

i = 0

for j in nans:
    print(df.columns[i],j)
    i = i+1

NU_INSCRICAO 0
NU_ANO 0
CO_MUNICIPIO_RESIDENCIA 0
NO_MUNICIPIO_RESIDENCIA 0
CO_UF_RESIDENCIA 0
SG_UF_RESIDENCIA 0
NU_IDADE 69
TP_SEXO 0
TP_ESTADO_CIVIL 0
TP_COR_RACA 0
TP_NACIONALIDADE 0
CO_MUNICIPIO_NASCIMENTO 145249
NO_MUNICIPIO_NASCIMENTO 145249
CO_UF_NASCIMENTO 145249
SG_UF_NASCIMENTO 145249
TP_ST_CONCLUSAO 0
TP_ANO_CONCLUIU 0
TP_ESCOLA 0
TP_ENSINO 2217073
IN_TREINEIRO 0
CO_ESCOLA 3947858
CO_MUNICIPIO_ESC 3947858
NO_MUNICIPIO_ESC 3947858
CO_UF_ESC 3947858
SG_UF_ESC 3947858
TP_DEPENDENCIA_ADM_ESC 3947858
TP_LOCALIZACAO_ESC 3947858
TP_SIT_FUNC_ESC 3947858
IN_BAIXA_VISAO 0
IN_CEGUEIRA 0
IN_SURDEZ 0
IN_DEFICIENCIA_AUDITIVA 0
IN_SURDO_CEGUEIRA 0
IN_DEFICIENCIA_FISICA 0
IN_DEFICIENCIA_MENTAL 0
IN_DEFICIT_ATENCAO 0
IN_DISLEXIA 0
IN_DISCALCULIA 0
IN_AUTISMO 0
IN_VISAO_MONOCULAR 0
IN_OUTRA_DEF 0
IN_GESTANTE 0
IN_LACTANTE 0
IN_IDOSO 0
IN_ESTUDA_CLASSE_HOSPITALAR 0
IN_SEM_RECURSO 0
IN_BRAILLE 0
IN_AMPLIADA_24 0
IN_AMPLIADA_18 0
IN_LEDOR 0
IN_ACESSO 0
IN_TRANSCRICAO 0
IN_LIBRAS 0
IN_TEMPO_ADIC

Observamos que as notas faltantes correnspondem aos alunos que faltaram no dia do vestibular ou tiveram sua prova anulada:

In [6]:
present = [
    df.groupby('TP_PRESENCA_CN').NU_NOTA_CN.sum().compute(),
    df.groupby('TP_PRESENCA_CH').NU_NOTA_CH.sum().compute(),
    df.groupby('TP_PRESENCA_LC').NU_NOTA_LC.sum().compute(),
    df.groupby('TP_PRESENCA_MT').NU_NOTA_MT.sum().compute(),]
    
print(present)


KeyboardInterrupt: 

 Há tabém alguns dados faltantes relacionados a esocola de conclusão do Ensino Médio. É óbivio que os alunos treneiros não concluiram o ensino básico não possuem essas informações. 
 
 * Veremos se há uma corelação entre os tipos de vestibulandos, segundo o dicionário dos microdados do ENEM temos os seguintes tipos na coluna TP_ST_CONCLUSAO (Situação de conclusão do Ensino Médio):
    - 1: Já concluiu o EM, 
    - 2: Concluiu o EM em 2019, 
    - 3: Vai concluir o EM após 2019 (treneiros), 
    - 4: não concluiu e não está cursnado o EM (treneiros).

In [ ]:
df.groupby('TP_ST_CONCLUSAO').CO_ESCOLA.mean().compute()

TP_ST_CONCLUSAO
1             NaN
2    3.126575e+07
3             NaN
4             NaN
Name: CO_ESCOLA, dtype: float64

De fato, apenas que concliu o Ensio Médio no ano do vestibular respondeu o questionário sobre a escola. Agora vou realizar uma análise relacionado ao tipo de escola para saber se há também perda de dados entre as escolas públicas e privadas. 
* Segundo os Microdados do ENEM a coluna TP_ESCOLA representa:
    - 1:	Não Respondeu	
    - 2:	Pública	
    - 3:	Privada	
    - 4:	Exterior	


### Análise corelacional dos dados.

In [ ]:
#Correlação entre as notas.


ENEM = df.iloc[:,91:111].compute()
ENEM.drop(["TP_STATUS_REDACAO"],axis=1)
ENEM.corr()

,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,TP_LINGUA,TP_STATUS_REDACAO,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO
NU_NOTA_CN,1.000000,0.669247,0.642877,0.648031,-0.256514,-0.108680,0.431610,0.412814,0.464822,0.440159,0.457992,0.505337
NU_NOTA_CH,0.669247,1.000000,0.745167,0.610630,-0.248580,-0.163584,0.466884,0.437826,0.490901,0.458775,0.478506,0.530075
NU_NOTA_LC,0.642877,0.745167,1.000000,0.589680,-0.265564,-0.186751,0.491372,0.440155,0.498979,0.464634,0.480892,0.538595
NU_NOTA_MT,0.648031,0.610630,0.589680,1.000000,-0.268104,-0.103768,0.426359,0.402118,0.453621,0.439886,0.445899,0.495625
TP_LINGUA,-0.256514,-0.248580,-0.265564,-0.268104,1.000000,0.041148,-0.165077,-0.141076,-0.159700,-0.150511,-0.157619,-0.175340
TP_STATUS_REDACAO,-0.108680,-0.163584,-0.186751,-0.103768,0.041148,1.000000,-0.640958,-0.508415,-0.508195,-0.567380,-0.273842,-0.538121
NU_NOTA_COMP1,0.431610,0.466884,0.491372,0.426359,-0.165077,-0.640958,1.000000,0.706343,0.784296,0.826921,0.571270,0.851560
NU_NOTA_COMP2,0.412814,0.437826,0.440155,0.402118,-0.141076,-0.508415,0.706343,1.000000,0.864486,0.742162,0.657294,0.893386
NU_NOTA_COMP3,0.464822,0.490901,0.498979,0.453621,-0.159700,-0.508195,0.784296,0.864486,1.000000,0.813357,0.708169,0.935611
NU_NOTA_COMP4,0.440159,0.458775,0.464634,0.439886,-0.150511,-0.567380,0.826921,0.742162,0.813357,1.000000,0.643937,0.893081


* Corelação entre o tipo de escola e as notas.
    - 1:	Não Respondeu	
    - 2:	Pública	
    - 3:	Privada	

In [ ]:
school = [
    df.groupby('TP_ESCOLA').NU_NOTA_CN.mean().compute(),
    df.groupby('TP_ESCOLA').NU_NOTA_CH.mean().compute(),
    df.groupby('TP_ESCOLA').NU_NOTA_LC.mean().compute(),
    df.groupby('TP_ESCOLA').NU_NOTA_MT.mean().compute(),
    df.groupby('TP_ESCOLA').NU_NOTA_REDACAO.mean().compute()]
    
print(school)

[TP_ESCOLA
1    479.563936
2    460.689326
3    540.503285
Name: NU_NOTA_CN, dtype: float64, TP_ESCOLA
1    509.139317
2    490.614694
3    567.779802
Name: NU_NOTA_CH, dtype: float64, TP_ESCOLA
1    522.200121
2    507.446984
3    565.609163
Name: NU_NOTA_LC, dtype: float64, TP_ESCOLA
1    522.746464
2    504.241441
3    619.760514
Name: NU_NOTA_MT, dtype: float64, TP_ESCOLA
1    569.752806
2    544.976565
3    722.038979
Name: NU_NOTA_REDACAO, dtype: float64]


As escolas particulares obteram melhores médias do que os alunos de escolas públicas e dos que não resolveram.

A próxima análise será das notas em relação a região.


In [ ]:
states = [
    df.groupby('CO_UF_RESIDENCIA').NU_NOTA_CN.mean().compute(),
    df.groupby('CO_UF_RESIDENCIA').NU_NOTA_CH.mean().compute(),
    df.groupby('CO_UF_RESIDENCIA').NU_NOTA_LC.mean().compute(),
    df.groupby('CO_UF_RESIDENCIA').NU_NOTA_MT.mean().compute(),
    df.groupby('CO_UF_RESIDENCIA').NU_NOTA_REDACAO.mean().compute()]
    
print(states)

[CO_UF_RESIDENCIA
11    459.223026
12    451.290462
13    450.881956
14    467.313946
15    457.205827
16    450.267287
17    458.528458
21    450.882586
22    459.247826
23    468.256969
24    475.584920
25    467.781141
26    470.560241
27    458.547809
28    468.779865
29    464.749375
31    492.333365
32    488.971497
33    491.892385
35    496.996064
41    492.350597
42    493.055076
43    486.504692
50    474.637703
51    467.056028
52    478.258151
53    492.786286
Name: NU_NOTA_CN, dtype: float64, CO_UF_RESIDENCIA
11    487.215906
12    480.995246
13    481.354141
14    494.990995
15    485.580461
16    482.029298
17    481.542105
21    477.666612
22    484.998358
23    497.715044
24    502.947521
25    495.907691
26    498.546074
27    485.499958
28    496.007553
29    493.379388
31    520.842738
32    517.651302
33    523.361916
35    527.785569
41    523.545363
42    525.569803
43    523.481137
50    500.194397
51    494.280161
52    505.836761
53    523.134671
Name: NU_NOTA

A partir do questionário socio-econômico do ENEM anazlisei a média das notas para cada uma das respostas.

In [6]:
# O processamento feito a seguir foi salvo no arquivo social-questionnaire.json pois demorou para calcular

'''
i = 1
questionnaire = []

while i <10:
    questionnaire.append({[
    df.groupby('Q00'+str(i)).NU_NOTA_CN.mean().compute().tolist(),
    df.groupby('Q00'+str(i)).NU_NOTA_CH.mean().compute().tolist(),
    df.groupby('Q00'+str(i)).NU_NOTA_LC.mean().compute().tolist(),
    df.groupby('Q00'+str(i)).NU_NOTA_MT.mean().compute().tolist(),
    df.groupby('Q00'+str(i)).NU_NOTA_REDACAO.mean().compute().tolist()]})
    i = i+1
    print("ok",i)

while i < 26:
    questionnaire.append([
    df.groupby('Q0'+str(i)).NU_NOTA_CN.mean().compute().tolist(),
    df.groupby('Q0'+str(i)).NU_NOTA_CH.mean().compute().tolist(),
    df.groupby('Q0'+str(i)).NU_NOTA_LC.mean().compute().tolist(),
    df.groupby('Q0'+str(i)).NU_NOTA_MT.mean().compute().tolist(),
    df.groupby('Q0'+str(i)).NU_NOTA_REDACAO.mean().compute().tolist()])
    i = i+1
    print("ok",i)

with open('social-questionnaire.json', 'w') as f:
    json.dump(questionnaire, f)

'''

# Lendo os dados que foram pré-processados

with open('social-questionnaire.json') as f:
   questionnaire = json.load(f)

print(questionnaire[0])

## III. Data Preparation

+ Select data:   
+ Clean data:   
+ Construct data/transform data/feature engineering:   
+ Integrate data:   
+ (Re) Format data:   

## IV. Modeling

+ Select modeling techniques: 
+ Generate test design: 
+ Build model: 
+ Assess model:  

## V. Evaluation                    

+ Evaluate results: 
+ Review process: 
+ Determine next steps: 

## VI. Deployment

+ Plan deployment: 
+ Plan monitoring and maintenance: 
+ Produce final report: 
+ Review project: 